Standard library import, change path to GOSTnets path

In [1]:
import os, sys
import pandas as pd
import networkx as nx
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn

peartree version: 0.6.1 
networkx version: 2.3 
matplotlib version: 3.0.3 
osmnx version: 0.9 


### Destination Prep
Here we seek to remove rows in the destination files which don't have valid Latitude / Longitude coordinate pairs, and standardize column names

Define path variable 

In [2]:
pth = r'C:\Users\charl\Documents\GOST\SierraLeone\Destinations'

Import schools dataset, remove rows which don't have both a valid Latitude and Longitude

In [3]:
df = pd.read_csv(os.path.join(pth, "schools.csv"))
df = df.loc[(df.LATITUDE.isna() == False) & (df.LONGITUDE.isna() == False)]
df.to_csv(os.path.join(pth, "schools.csv"))

C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,15,34,82,521) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Import health center dataset, remove rows which don't have both a valid Latitude and Longitude, rename columns to be consistent with schools dataset

N.B. commented out, as it won't work if run more than once - columns will cease to exist!

In [5]:
df = pd.read_csv(os.path.join(pth, "health_centers.csv"), encoding = 'utf-8')
#df = df.loc[(df.Lat.isna() == False) & (df.Long.isna() == False)]
#df = df.rename({'Lat':'LATITUDE','Long':'LONGITUDE'}, axis = 1)
df.to_csv(os.path.join(pth, "health_centers.csv"), encoding = 'utf-8')

### Find Largest Graph

Here, we generate shapefiles of the connected network and those roads which are disconnected. Though not necessary for the analysis, this is a useful process to go through to:
1.) visually appraise the quality of the OSM network
2.) identify large subgraphs that need to be manually connected to the main network
3.) support network improvement activities 

Import Processed Graph

In [6]:
net_pth = r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet'
G = nx.read_gpickle(os.path.join(net_pth, 'SLE_processed.pickle'))

Add a unique value to every edge

In [7]:
q = 0
for u, v, data in G.edges(data = True):
    data['unique_id'] = q
    q+=1

Identify largest subgraph by making a list of all subgraphs, iterating through them, and setting a variable to the maximum

In [8]:
list_of_graphs = list(nx.strongly_connected_component_subgraphs(G))

In [9]:
max_edges = 0
for q in range(0, len(list_of_graphs)):
        g = list_of_graphs[q]
        if g.number_of_edges() > max_edges:
            max_edges = g.number_of_edges()
            t = q
        else:
            pass

Print the results of this test

In [10]:
print("The largest graphs is the graph in position %s, and has %s edges" % (t, max_edges))

The largest graphs is the graph in position 0, and has 139617 edges


Define largest graph as its own object, save down

In [12]:
largest_G = list_of_graphs[t]
gn.save(largest_G, 'largest_G', r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet')
edge_gdf_largest_G = gn.edge_gdf_from_graph(largest_G)

# Turn into shapefile
edge_gdf_largest_G = edge_gdf_largest_G.drop('geometry', axis = 1)
edge_gdf_largest_G['Wkt'] = edge_gdf_largest_G['Wkt'].apply(lambda x: gn.unbundle_geometry(x))
edge_gdf_largest_G = edge_gdf_largest_G.set_geometry('Wkt')
edge_gdf_largest_G.to_file(os.path.join(r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet','LargestG.shp'))

Create a shapefile of all the edges that aren't in the main graph

In [13]:
edge_gdf = gn.edge_gdf_from_graph(G)

edges_in_largest_G = list(edge_gdf_largest_G.unique_id)

edges_NOT_in_largest_G = edge_gdf.loc[~edge_gdf.unique_id.isin(edges_in_largest_G)]

Save it down

In [14]:
edges_NOT_in_largest_G = edges_NOT_in_largest_G.drop('geometry', axis = 1)
edges_NOT_in_largest_G['Wkt'] = edges_NOT_in_largest_G['Wkt'].apply(lambda x: gn.unbundle_geometry(x))
edges_NOT_in_largest_G = edges_NOT_in_largest_G.set_geometry('Wkt')
edges_NOT_in_largest_G.to_file(os.path.join(r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet','DisconnectedRoads.shp'))